## Content
[1. Introduction ](#1)

[2. Libraries](#2)
  * [2.1 Insights & Summary](#2.11)

[3. Feature Engineering](#3)
  * [3.1 ](#3.1)

[4. Exploratory Data Analysis ](#3)
  * [4.1 Question 1](#3.1)
  * [4.2 Question 2](#3.2)
  * [4.3 Question 3](#3.3)


<a id='#1'></a>
**ShopSmart Inc.** is an e-commerce platform that sells a variety of products online. The company aims to optimize its website and enhance customer satisfaction by understanding customer behavior and preferences. 

To achieve this goal, ShopSmart Inc. plans to analyze the data collected from its website to gain insights into customer interactions, purchasing patterns, and product preferences.


Project Objective: The objective of this project is to utilize data from ShopSmart Inc.'s website to understand customer behavior and preferences. By analyzing the data, the company aims to:
1.     Segment customers based on their preferences, geographic location, and purchasing behavior.
2.     Enhance website usability and user experience based on insights gained from customer interactions.
3.     Optimize product offerings and marketing strategies to improve customer satisfaction and retention.


## 1. Libraries 
<a id="1"></a>

In [ ]:
# !pip install a-world-of-countries
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import missingno
import plotly.graph_objects as go
import plotly.express as px
import awoc
import seaborn as sns
import warnings
from  IPython.display import display,Markdown

In [5]:
warnings.filterwarnings('ignore')
def printmd(value):
    display(Markdown(value))
    
sns.set_theme(style="white", palette="pastel")

sns.set_palette('dark')

def get_continent(location):
    for obj in all_countries:
        if obj['Country Name'] == location:
            return obj['Continent Name']
    else:
        if location == 'Korea':
            return 'Asia'
        elif location == 'United States of America':
            return 'North America'
        return 'Other Islands'
        
def added_removed(feature_name1):
    product_added = pd.pivot(data=new_data[new_data['event_type'] == 'add_to_cart'], values=feature_name1,columns='continent').mode().stack()[0]
    product_removed = pd.pivot(
        data=new_data[new_data['event_type'] == 'remove_from_cart'],
        values=feature_name1, columns='continent').mode().stack()[0]
    product_result = pd.concat([
        product_added,product_removed],axis=1, 
        ignore_index=True,names=['added to cart','removed from cart'])
    
    printmd('Each **CONTINENT** and the product customers **ADDED TO AND REMOVED FROM CART**:\n')
    display(product_result.reset_index().rename(columns={'level_0': 'continent', 0: 'added to cart',1:'removed from cart'}))
    print('')

def get_max_n_min_product(feature,feature_name):
    brand_max =  new_data[new_data['event_type'] == feature]['brand'].mode()[0]
    brand_min = new_data[new_data['event_type'] == feature]['brand'].value_counts().idxmin()
    res_max = new_data[new_data['event_type'] == feature]['product_name'].mode()[0]
    res_min = new_data[new_data['event_type'] == feature]['product_name'].value_counts().idxmin()
    
    printmd('The ***most {} brand*** is **{}**'.format(feature_name,brand_max))
    printmd('The ***least {} brand*** is **{}**'.format(feature_name,brand_min))
    print('')
    printmd('The ***most {} product*** is **{}**'.format(feature_name,res_max))
    printmd('The ***least {} product*** is **{}**'.format(feature_name,res_min))

def unlock_df(df_name):
    new_data = df_name['event_data'].apply(json.loads)
    event_df = pd.DataFrame(item for item in new_data)
    return event_df

def visited_checkout_product(feature_name1):
    product_visit = pd.pivot(data=event_visit_type, values=feature_name1, columns='continent').mode().stack()[0]
    product_checkout = pd.pivot(data=checkedout, values=feature_name1, columns='continent').mode().stack()[0]
    product_result = pd.concat([product_visit,product_checkout],axis=1, ignore_index=True,names=['visit','checkout'])
    
    printmd('Each **CONTINENT** and the product customers **VISITED THE MOST**:\n')
    display(product_result.reset_index().rename(columns={'level_0': 'continent', 0: 'most visited brand',1:'most checked out'}))
    print('')
    printmd('Three brands seem to be the most visited in the continents. More of these brands should be made available to the continents and countries.')

def get_percentages(d_series):
    return round(d_series/d_series.sum()*100,2)

NameError: name 'warnings' is not defined

#### import files

In [ ]:
customers = pd.read_csv('customers.csv')
events = pd.read_csv('events.csv')
line_items = pd.read_csv('line_items.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [ ]:
customers.info()

In [ ]:
products.info()

In [ ]:
events.info()

In [ ]:
orders.info()

In [ ]:
line_items.info()

In [ ]:
# Merge the customers, events, line_items, orders, and products tables on the customer_id column
merged_data = pd.merge(customers, events, on='customer_id', how='outer') # merged customers and events
merged_data = pd.merge(merged_data, orders, on='customer_id', how='outer') # merged above and customer_id
merged_data = pd.merge(merged_data, line_items, on='order_id', how='outer') # merged above and order_id
merged_data = pd.merge(merged_data, products, left_on='item_id', right_on='id', how='outer') # merged above and item_id

merged_data.info()

In [ ]:
merged_data.rename(columns={'name':'product_name', 'location':'country'},inplace=True) # rename some of the column names

In [ ]:
merged_data.info()

In [ ]:
new_data = pd.concat([merged_data,unlock_df(merged_data).drop(['status','quantity','order_id','item_id'], axis=1)],axis=1).drop(['event_data','item_id','id','line_item_id','event_timestamp'],axis=1)
new_data.head()

In [ ]:
printmd('**Missing Value Percentages**')
(new_data.isna().sum()/new_data.shape[0]*100).sort_values(ascending=False)

In [ ]:
new_data['quantity'] = new_data['quantity'].fillna(0).astype(int)

In [ ]:
new_data.describe()

In [ ]:
new_data.describe(exclude='number').T

In [ ]:
printmd(f'The number of duplicate rows = {len(new_data[new_data.duplicated()])}')

In [ ]:
missingno.heatmap(new_data)

In [ ]:
new_data['currency'] = new_data['currency'].astype('category')

### Date Time Transformation

In [ ]:
new_data['timestamp'] = pd.to_datetime(new_data['timestamp'],format='%Y-%m-%dT%H:%M:%S.%f')
new_data['checked_out_at'] = pd.to_datetime(new_data['checked_out_at'], format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
printmd(f'Dataset start date = {new_data["timestamp"].dt.date.min()}')
printmd(f'Dataset end date = {new_data["timestamp"].dt.date.max()}')
printmd(f'Dataset Spanning Through {new_data["timestamp"].dt.date.max() - new_data["timestamp"].dt.date.min()}')

In [ ]:
new_data['day'] = new_data['timestamp'].dt.day_name()

### Feature Engineering
<a id=2></a>

In [ ]:
country = new_data['country']
country = country.apply(lambda x: x.split('(')[0].strip())
# Replace country names so as to get them from awoc library
country.replace({'United States of America': 'United States',
                     'Congo': 'Democratic Republic of the Congo',
                     "Lao People's Democratic Republic":'Laos',
                     "Brunei Darussalam":'Brunei',
                     "Kyrgyz Republic":'Kyrgyzstan',
                     "Macao":'Macau',
                     "Palestinian Territory":"Palestine",
                     "Syrian Arab Republic":'Syria',
                     "Cote d'Ivoire":'Ivory Coast',
                     'Russian Federation':'Russia',
                     "North Macedonia":'Macedonia', 
                     'South Korea':'Korea',
                     'North Korea':'Korea',
                     'Cocos':'Cocos Islands',
                     "Libyan Arab Jamahiriya":'Libya'}, inplace=True)


In [ ]:
all_countries = awoc.AWOC().get_countries() # get dictionary of all countries



In [ ]:
new_data['continent'] = country.apply(get_continent)
new_data['country'].replace({'Democratic Republic of the Congo':'Congo'},inplace=True)

In [ ]:
new_data['price_range'] = pd.qcut(new_data.price,3, labels=['low','mid','high']) # separating prices into three price range 

In [ ]:
new_data['product_name'].replace({'iPhone 13':'Apple iPhone 13'}, inplace=True) # Added brand name to iPhone

In [ ]:
new_data['brand'] = new_data['product_name'].apply(lambda x : x.split(' ')[0] if not pd.isna(x) else x) # extract brand name from products

In [ ]:
new_data['number_of_days_transaction'] = np.ceil(new_data.groupby('customer_id')['timestamp'].\
                                                  transform(lambda x:np.ptp(x)).dt.total_seconds()/86400) #86400 is the total seconds in a day

In [ ]:
new_data['total_order_price'] = new_data['quantity'] * new_data['price']

In [ ]:
ax = new_data['price_range'].value_counts().plot(kind='bar',figsize=(15,5))
plt.xticks(rotation=0)
plt.xlabel('Price Range')
plt.title('Price Range Count')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', linewidth=0.5)

## 3. EDA 
<a id="3"></a>

In [ ]:
printmd('Of all transaction performed, **total of customers** are **{1}** <br> **Total devices** used on the sites are **{0}** <br>**Total transactions** are **{2}**'
        .format(new_data['device_id'].nunique(),new_data['customer_id'].nunique(),new_data['order_id'].nunique()))

In [ ]:
# price distribution
ax = sns.histplot(new_data['price'],bins=4)
ax.set_title('Price Distribution')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
# Revenue Distribution
plt.figure(figsize=(15,5))
ax = sns.histplot(data = new_data['total_order_price'],bins=20)
ax.set_title('Total Order Distribution')

The

In [ ]:
corr_features = ['quantity','price','number_of_days_transaction']
new_data[corr_features].corr()

##### There are just little correclation between the price, the quantity checked out and number of days of transaction

In [ ]:
# correlation between price and quantity of product bought
sns.scatterplot(data=new_data[new_data['event_type'] == 'checkout'], x='quantity',y='price', hue='status')
printmd('There is no difference in the price and the quantity of products checked out')

In [ ]:
sns.barplot(new_data['event_type'].value_counts())

From the figure above, <b>Africa</b> has the <b>highest number of transaction</b> which may be due to the fact that Africa has the highest number of countries, followed by <b>Asia</b> and <b>Antarctica</b> is the <b>lowest</b>

### 3.1 Visit Frequency 
<!-- <a id="3.1"></a> -->
- How often does this customer visit the website, based on the provided data?
- Can you identify any patterns or trends in the customer's visit frequency?


In [ ]:
#get the event type which are equal to visit
event_visit_type = new_data[new_data['event_type'] == 'visit']
event_visit_type.head()

In [ ]:
visit_day_number = event_visit_type.groupby('number_of_days_transaction')['customer_id'].nunique()

percentages = get_percentages(visit_day_number)
fig = px.bar(x=visit_day_number.index,y=visit_day_number.values,text=percentages.apply(lambda x :str(x) + '%'))

fig.update_layout(title='Number of visits days by customers',xaxis_title='Number of Days of Transaction',yaxis_title='Count')
fig.show()

In [ ]:
printmd(f"**{percentages.iloc[0]}%** of the customers visit the website for **just one day** while it can be seen that there is an increase in some customers visit after **day 2** till **day 11** after which there is almost absolute drop to zero. \nThe means that customers **if they dont finish their transaction on same day may take up to 11 days to** to make decision on the products they want to purchase")

In [ ]:
ax = event_visit_type['timestamp'].dt.day_name().value_counts().sort_values(ascending=False).plot(kind='bar',figsize=(15,5))
plt.title('Daily Customers interaction with websites')
plt.xticks(rotation=0)
plt.xlabel('Day of the week')
plt.ylabel('Customer Visit count')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', linewidth=0.5)
plt.show()
printmd('The website is <b>VISITED</b> mostly on <b>SATURDAY</b> while <b>SUNDAY</b> has the <b>LEAST VISIT</b>')

In [ ]:
ax = event_visit_type['timestamp'].dt.hour.plot(kind='hist',bins=4)
mode_hour = new_data['timestamp'].dt.hour.mode()[0]
plt.axvline(mode_hour, color='b', linestyle='--', label='Mode(Hour)')
plt.legend()
ax.set_xlabel('Hours')
plt.show()

printmd('Customers visit the websites mostly during the <b>FIRST QUARTER</b> of the day which is between <b>12 AM</b> to <b>6 AM</b> and the **highest visit** being <b>4 AM</b>')

In [ ]:
product_visit = pd.pivot(data=event_visit_type, values='product_name', columns='country').mode().stack()[0]
printmd('Each <b>COUNTRY</b> and the product customers <b>VISITED THE MOST</b>:\n')
product_visit.reset_index(name='most visited product')

In [ ]:
visit_status = event_visit_type.groupby('status')['customer_id'].nunique()
fig = go.Figure(data=go.Pie(labels=visit_status.index, values=visit_status, hole=0.5))
fig.update_layout(title = 'Visit with Checkout Status of customers')
fig.show()

In [ ]:
printmd('Out of the **1000 customers**, **successful transaction (32.7%)**, **failed transaction (33.6%)** while **cancelled (33.7%)** ')
printmd('The likelihood of a customer visiting and successfully purchasing a product is slim as the amount of failed and cancelled are more than the number of success')

In [ ]:
visit_country = event_visit_type.groupby('continent')['customer_id'].nunique().sort_values(ascending=False)
percentages = get_percentages(visit_country)
fig = go.Figure(data=go.Bar(x=visit_country.index, y=visit_country.values, text=percentages.apply(lambda y :str(y) + '%')))
fig.update_layout(title = 'Visit with Status count of customers')
fig.show()

In [ ]:
printmd(f'Most of the visits to the website are customers from **{percentages.index[0]} ({percentages.values[0]}%)**, **{percentages.index[1]} ({percentages.values[1]}%)** and **{percentages.index[2]} 
({percentages.values[2]}%)** making the **top 3** of the continents that visits the website')

printmd("Customers from **Africa**, are the **highest visitor**, followed by Asian customers and Europe, the place where customers are least is the Antarctica, This is expected as the amount of counrties in Africa are more than than Asia and Asia are more than others")

#### Most visited brand

In [ ]:
brand_customers_visit = event_visit_type.groupby('brand')
['customer_id'].nunique().sort_values(ascending=False)
plt.figure(figsize=(15,5))
ax = sns.barplot(brand_customers_visit)
ax.set_title('Brands as were visited by customers')
ax.set_ylabel('Customers count')
plt.show()

In [ ]:
printmd('During the customers visit, **Apple, Sony and Samsung** brands are the **most interacted with**. As such it is advised that ShopSmart get **more products from these brands** can generate more revenue')

In [ ]:
# Percentage Completion Rate
checkedout = new_data[(new_data['event_type'] == 'checkout')]

status_counts = checkedout['status'].value_counts(dropna=False)

fig = go.Figure(data=go.Pie(
    labels=status_counts.index,
    values=status_counts, hole=0.5))


fig.update_layout(title='Customer\'s Transaction Checkout Status Rate')

fig.show()


In [ ]:
printmd("There is **no correlation** between the **quantity of product** purchased and the **price of the products**")  

In [ ]:
printmd(f'The time for **most checkout** is within **{checkedout["checked_out_at"].dt.hour.value_counts().index[0]} AM**')
# printmd()

Customers may wish to purchase products that are the most visited in their country or continents

In [ ]:
printmd('Each <b>CONTINENT</b> and the product customers <b>VISITED THE MOST</b>:\n')
display(product_visit.reset_index(name='most visited product'))
print('')
printmd('Three brands seems to be the most visited in the continents, more of these brands should be made available to the continents and countries')

In [ ]:
product_visit = pd.pivot(data=event_visit_type, values='brand',
                         columns='continent').mode().stack()[0]
product_checkout = pd.pivot(data=checkedout, values='brand',
                            columns='continent').mode().stack()[0]
product_result = pd.concat([product_visit,product_checkout],axis=1,
                           ignore_index=True,names=['visit','checkout'])

printmd('Each **CONTINENT** and the product customers **VISITED THE MOST**:\n')
display(product_result.reset_index().rename(columns={'level_0': 'continent', 0: 'most visited brand',1:'most checked out'}))
print('')
printmd('Three brands seem to be the most visited in the continents. More of these brands should be made available to the continents and countries.')

<a id="3.2"></a>
### 3.2 Location Analysis:
- What is the customer's location based on the provided data?
- How might the customer's location influence their purchasing behavior?


#### customers location

In [ ]:
ax = new_data.groupby('continent')['status'].value_counts().unstack().sort_values(by=['failed','cancelled','success'], ascending=False).plot(kind='bar', stacked=True, figsize=(15,5))
plt.xticks(rotation=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', linewidth=0.5)
plt.title('Status of transaction for Continent')
plt.show()

In [ ]:
new_data_location = new_data.groupby(['country','continent'])['customer_id'].nunique().sort_values(ascending=False).reset_index(name='count')
plt.figure(figsize=(15,5))
sns.barplot(new_data_location[:10], x='country', y = 'count', hue='continent')
plt.title('Top number of customers in country')
plt.show()
new_data_location['percentages'] = new_data_location['count'].div(new_data_location['count'].sum())*100

display(new_data_location)

In [ ]:
new_data_location = new_data.groupby(['continent'])['customer_id'].nunique().sort_values(ascending=False).reset_index(name='count')
plt.figure(figsize=(15,5))
sns.barplot(new_data_location[:10], x='continent', y = 'count', hue='continent')
plt.title('Top number of customers in continent')
plt.show()
new_data_location['percentages'] = new_data_location['count'].div(new_data_location['count'].sum())*100

display(new_data_location)

In [ ]:
printmd(f"Out of the **{new_data['country'].unique().shape[0]}** Countries, **22.72%** of them are **Africans**, followed by **Asia** with **21.49%** of the customers while **Antarctica (0.36)%** which is the least")

#### Successful Order by Continent

In [ ]:
# order by continent
plt.figure(figsize=(15,5))
result = new_data[new_data['status'] == 'success'].groupby('continent')['order_id'].nunique().sort_values(ascending=False)[:10]
percentages = get_percentages(result)
fig = px.bar(x=result.index,y=result.values,text=percentages.apply(lambda x :str(x) + '%'))

fig.update_layout(title='Successful Order by Continent',xaxis_title='Continent',yaxis_title='Count')
fig.show()

In [ ]:
printmd('Africa has the most Successful Order followed by Asia and Europe')
printmd('<b>AFRICAN CUSTOMERS (23.42%)</b> visits the website the most and there are <b>high number of CHECKOUT</b> for customers in <b>AFRICA</b>, <b>EUROPE (21.41%)</b>, and <b>ASIA (19.81%)</b>')

In [ ]:
# order by continent
plt.figure(figsize=(15,5))
result = new_data[new_data['status'] == 'success'].
groupby('country')['order_id'].nunique().
sort_values(ascending=False)[:10]
percentages = get_percentages(result)
fig = px.bar(x=result.index,y=result.values,
             text=percentages.apply(lambda x :str(x) + '%'))

fig.update_layout(title='Top 10 Successful Order by Country',xaxis_title='Country',yaxis_title='Count')
fig.show()

<b>AFRICAN CUSTOMERS (23.42%)</b> visits the website the most and there are <b>high number of CHECKOUT</b> for customers in <b>AFRICA</b>, <b>EUROPE (21.41%)</b>, and <b>ASIA (19.81%)</b>
 

#### Places where order comes from

In [ ]:
pd.pivot_table(data=new_data,columns='status',
               index='continent',aggfunc='count',
               values='product_name')

In [ ]:
res = new_data.groupby(['country','event_type']).size().unstack()
res_total = res.sum(1).sort_values(ascending=False)
res = res.reindex(res_total.index)


ax = res[:10].plot(kind='bar', stacked=False,figsize=(15,5)) #plot top 10 locations and their status
plt.xlabel('Location')
plt.ylabel('Count')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.title('Status Counts for Top 10 Locations')
plt.legend(title='Status')
plt.show()

In [ ]:
ax = new_data.groupby('country')['status'].value_counts().unstack().sort_values(by=['success','cancelled','failed'], ascending=False)[:10]\
.plot(kind='bar', figsize=(15,5))
plt.xticks(rotation=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(axis='y', linestyle='--', linewidth=0.5)
plt.title('Top 10 Status of transaction for Countries ')
plt.show()

#### Most Added to cart and Removed Product

In [ ]:
added_removed('brand')
added_removed('product_name')

In [ ]:
printmd('Three brands seem to be the most visited in the continents. More of these brands should be made available to the continents and countries.')

In [ ]:
continent_status_rate = pd.pivot_table(
    data=new_data[new_data['event_type'] == 'checkout'], 
    columns='continent', values='quantity',index = 'status',aggfunc='sum')
continent_status_rate

In [ ]:
# Calculate percentages
total_counts = continent_status_rate.sum(axis=0)
percentages = (continent_status_rate.div(total_counts, axis=1) * 100).round(2)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=continent_status_rate.values,
    x=percentages.columns,
    y=percentages.index,
    colorscale='oryel',
    colorbar=dict(title='Percentage')
))

for i in range(len(percentages.index)):
    for j in range(len(percentages.columns)):
        fig.add_annotation(
            x=percentages.columns[j],
            y=percentages.index[i],
            text=str(percentages.values[i, j]) + '%',
            showarrow=False,
            font=dict(color='white'))

# Update layout
fig.update_layout(
    title='Successsfuly Ordered Product Quantity by Continent',
    xaxis_title='Continent',
    yaxis_title='Status'
)

# Show figure
fig.show()


**Successful** - Based on Location **Oceania** has the most purchase quantity followed by **Asia** and **Europe**\
**Cancelled** - The most are from **North America**, **Other Islands** and **South America**\
**Failed** - The failed purchased quantity comes from **Africa**, **Asia** and **Europe** 

In [ ]:
location_by_price = new_data.groupby('country')['total_order_price'].sum().sort_values(ascending=False)
plt.figure(figsize=(15,5))
sns.barplot(location_by_price[:10])

In [ ]:
ax = new_data.groupby(['country'])['price_range'].value_counts().unstack().sort_values(by=['low','mid','high'],ascending=[False,False,False])[:10].plot(kind='bar', figsize=(15,5))
plt.xticks(rotation=0)
plt.title('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend(title='Price Range')
plt.show()
print('Most Customers buy mid-range Price Products except for some Customers in countries like Switzerland who purchases low-range priced products')

### Overall Purchase Behavior: 
<a id='3.3'></a>

- Based on the data provided, what insights can you draw about this customer's overall behavior on the website?

- How might these insights inform marketing strategies or personalized recommendations for this customer?


#### currency and ease of payment

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.barplot(data=new_data.groupby(['currency', 'status']).size().reset_index(), x='currency', y=0, hue='status')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel('Currency')
plt.ylabel('Count')
plt.title('Status Counts by Currency')
plt.legend(title='Status')
plt.show()


Though the difference on some of the currency used in transaction is not quite much for GBP and NGN, NAIRA shows to be the most consistent followed by GBP and USD the last.

#### Removed From Cart

In [ ]:
removed_from_cart = new_data[new_data['event_type'] == 'remove_from_cart'].groupby(['product_name'])['customer_id'].nunique().sort_values()
printmd(f'**Total Removed Product = {removed_from_cart.sum()}**')
printmd('**Removed from Cart Percentage:**')
removed_from_cart/removed_from_cart.sum()*100

In [ ]:
printmd('The most removed from cart Product from is **fitbit charge 4(6.87%)** and the least removed is **Nintendo Switch (6.4%)**' )

#### Items Based on Event Type

In [ ]:
get_max_n_min_product('visit','visited')

In [ ]:
get_max_n_min_product('add_to_cart','added to cart')

In [ ]:
get_max_n_min_product('remove_from_cart','removed from cart')

In [ ]:
get_max_n_min_product('checkout','checked out')

##### The most visted items is also the most added, most removed and Checked out, Customers are more likely to visit the Canon EOS R5 Camera

The **most visited brands** are **Apple** and it the **most checked out** brand but are not the most checked out Product

It is obvious that Customers love purchasing the **Canon EOS R5 Camera** product from the **Canon** brand but **Apple** Products still remains the lead in the **Most Checked Out Brand**

#### Countries with most visit, goods bought

In [ ]:
successful_checkedout_country_total = new_data[(new_data['status'] == 'success') & (new_data['event_type'] == 'checkout')].groupby(['country','continent'])[['total_order_price']].sum().sort_values(by='total_order_price', ascending=False).reset_index()
successful_checkedout_country_total

In [ ]:
plt.figure(figsize=(20,5))
sns.barplot(data=successful_checkedout_country_total[:10],x='country',y='total_order_price',hue='continent')
plt.xlabel('Total Revenue')
plt.title('Top 10 Countries in Revenue')

#### Customers Order

In [ ]:
# looking at the status of customers that checked out

res = new_data[new_data['order_id'].notna()].groupby('customer_id')['status'].unique().value_counts()

fig = go.Figure(data=go.Pie(labels=res.index, values=res, hole=0.5))


fig.update_layout(title='Customer\'s Transaction Checkout Status Rate')

fig.show()

printmd('Out of <b>9165</b> customers that ordered, customers that <b>cancelled</b> order are <b>33.7%</b>, the percentage of <b>33.6% failed</b> transactions and the percentage of <b>33.6% successful</b> order')


In [ ]:
checkedout_only = new_data[(new_data['event_type'] =='checkout')]
result = checkedout_only.groupby(['number_of_days_transaction','status'])['total_order_price'].mean().unstack()
result.plot(kind='line',figsize=(15,5))
# plt.xlabel('Revenue')
# plt.ylabel('NUmber of days')
# plt.title('Number of days required to complete transaction by customers')

In [ ]:
plt.figure(figsize=(15,5))
most_revenue_by_brand = new_data[(new_data['status'] == 'success') & (new_data['event_type'] == 'checkout')].groupby('brand')['total_order_price'].sum().sort_values(ascending=False)
sns.barplot(data=most_revenue_by_brand)
plt.title('Revenue by brand')
plt.ylabel('Total Revenue')

**Canon** is the brand that **generate the most revenue** followed by ***Samsung*** while **Starbucks and Coca-Cola**
are the least revenue generating brands

Many Customers who use than an average of **2 days** complete their transaction successfully  
Transactions on **day 4** to **day 6** are likely to get **cancelled or fail**  
Transactions of more than **2 days** should be reminded Customers about their purchase so they dont forget

#### Successfully Checked out Products and Revenue

In [ ]:
result = checkedout_only[checkedout_only['status'] == 'success'].groupby('product_name')['total_order_price'].sum().sort_values(ascending=False)
percentages = get_percentages(result)
fig = px.bar(x=result.index, y=result.values, text=percentages.apply(lambda y :str(y) + '%'), height=400, width=900)
fig.update_layout(title='Successfully Purchased Product and Their Revenue')
fig.update_yaxes(title_text='Revenue')
fig.update_xaxes(title_text='Product')
fig.show()


In [ ]:
result = checkedout_only.groupby(['continent'])['number_of_days_transaction'].mean().reset_index().sort_values('number_of_days_transaction', ascending=False)
display(result)
plt.figure(figsize=(15,5))
sns.barplot(result,x='continent',y='number_of_days_transaction')

In [ ]:
result = checkedout_only.groupby(['country','continent'])['number_of_days_transaction'].mean().reset_index().sort_values('number_of_days_transaction', ascending=False)
result
fig, axes = plt.subplots(2,1, figsize=(15,5), sharey=True)
axes = axes.flatten()
sns.barplot(result[:10],x='country',y='number_of_days_transaction', hue='continent',ax=axes[0],legend=False).set_title('Top 10')
sns.barplot(result.tail(10),x='country',y='number_of_days_transaction', hue='continent',ax=axes[1],legend=False).set_title('Last 10')
plt.suptitle('Number of days by Customers to Complete Purchase')
plt.tight_layout()
axes[0].set_ylabel('Number of Days')
axes[1].set_ylabel('Number of Days')

It is observed that **Isreali , Sri Lankans (Asia), Slovenia(Europe), Libya(Africa)** takes the highest average of Days  
**Qatar(Africa), Svalbard andJan Mayen Islands(Other Islands), Maldives(Asia)** takes the least days to complete purchase

Creating of **discount/vouchers** which will help them to complete transactions should be introduced,
Customers from Continents and Countries that **take less time** should be able to **improve their service** e.g. speeding up support incase there is a required service from the support department can help keep these customers in making purchase in little timeframe